<a href="https://colab.research.google.com/github/MayCXC/NSF-REU-2021/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

if not os.path.exists('Xtst.csv'):
  !gdown --id 1wQDp91LC0PBJvJRBTLw1aW51-UN5JOm8
if not os.path.exists('Ytr.csv'):
  !gdown --id 1ZhPVjXlOttrYRjSRm_IUBZj-Bokswzbn
if not os.path.exists('Ytst.csv'):
  !gdown --id 10LE7A7yhNkPlk8Zv3dmBjFYwMW9ZKy9_
if not os.path.exists('Xtr.csv'):
  !gdown --id 1IUcv_OcLX1YKUB1Xw_lYiBK6bnyLxikS
if not os.path.exists('cnn1d.zip'):
  !curl --remote-name -H 'Accept: application/vnd.github.v3.raw' --location 'https://github.com/MayCXC/NSF-REU-2021/raw/main/cnn1d.zip'
#if not os.path.exists('discriminator.zip'):
#  !curl --remote-name -H 'Accept: application/vnd.github.v3.raw' --location 'https://github.com/MayCXC/NSF-REU-2021/raw/main/discriminator.zip'

Downloading...
From: https://drive.google.com/uc?id=1wQDp91LC0PBJvJRBTLw1aW51-UN5JOm8
To: /content/Xtst.csv
308MB [00:02, 129MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZhPVjXlOttrYRjSRm_IUBZj-Bokswzbn
To: /content/Ytr.csv
25.7MB [00:00, 149MB/s]
Downloading...
From: https://drive.google.com/uc?id=10LE7A7yhNkPlk8Zv3dmBjFYwMW9ZKy9_
To: /content/Ytst.csv
6.42MB [00:00, 99.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IUcv_OcLX1YKUB1Xw_lYiBK6bnyLxikS
To: /content/Xtr.csv
1.23GB [00:12, 97.7MB/s]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   134  100   134    0     0    482      0 --:--:-- --:--:-- --:--:--   482
100 1232k  100 1232k    0     0  2107k      0 --:--:-- --:--:-- --:--:-- 2107k


In [2]:
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

Xtr = pd.read_csv("Xtr.csv", sep=',', header=None)
Xtst = pd.read_csv("Xtst.csv", sep=',', header=None)
Ytr = pd.read_csv("Ytr.csv", sep=',', header=None)
Ytst = pd.read_csv("Ytst.csv", sep=',', header=None)

def train_test_base_model(Xtr, Xtst, Ytr, Ytst, epochs):
  scaler = preprocessing.StandardScaler().fit(Xtr)
  xtr = scaler.transform(Xtr)  
  xtst = scaler.transform(Xtst)  

  xtr = xtr.reshape(xtr.shape[0],xtr.shape[1],1)
  xtst = xtst.reshape(xtst.shape[0],xtst.shape[1],1)

  print ("Number of training samples is: ",xtr.shape[0])
  print ("Number of test samples is: ",xtst.shape[0])

  num_classes = 2

  ytr = to_categorical(Ytr, num_classes)
  ytst = to_categorical(Ytst, num_classes)

  model = Sequential()
  model.add(Conv1D(32, kernel_size=7, activation='relu', input_shape=xtr.shape[1:]))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))    
  model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
  model.fit(xtr, ytr, epochs=epochs, verbose=2, batch_size=128,validation_split=0.3)  #learning_rte
  print("Evaluation of best performing model:")
  model.evaluate(xtst, ytst)
  return model

if not os.path.exists('cnn1d.zip'):
  model = train_test_base_model(Xtr,Xtst,Ytr,Ytst,150)
  model.save('cnn1d')
  !zip -r cnn1d.zip cnn1d
else:
  if not os.path.exists('cnn1d'):
    !unzip cnn1d.zip
  model = load_model('cnn1d')

Archive:  cnn1d.zip
   creating: cnn1d/
   creating: cnn1d/variables/
  inflating: cnn1d/variables/variables.index  
  inflating: cnn1d/variables/variables.data-00000-of-00001  
   creating: cnn1d/assets/
  inflating: cnn1d/saved_model.pb    
  inflating: cnn1d/keras_metadata.pb  


In [15]:
import numpy as np

def percentiles(x,y,p):
  benign = np.compress(y == 0.0, x, axis=0)
  totals = np.sum(benign, axis=1)
  return (y == 0.) & (np.sum(x, axis=1) < np.percentile(totals,p))

Ytr_low = percentiles(Xtr.to_numpy(),Ytr.to_numpy().flatten(),10).astype(int)
Ytst_low = percentiles(Xtst.to_numpy(),Ytst.to_numpy().flatten(),10).astype(int)

if not os.path.exists('generator.zip'):
  generator = train_test_base_model(Xtr, Xtst, Ytr_low, Ytst_low, 10)
  generator.save('generator')
  !zip -r generator.zip generator
else:
  if not os.path.exists('generator'):
    !unzip generator.zip
  generator = load_model('generator')

Number of training samples is:  1029085
Number of test samples is:  256883
Epoch 1/10
5628/5628 - 52s - loss: 0.1099 - accuracy: 0.9524 - val_loss: 0.1319 - val_accuracy: 0.9463
Epoch 2/10
5628/5628 - 51s - loss: 0.0901 - accuracy: 0.9604 - val_loss: 0.1459 - val_accuracy: 0.9484
Epoch 3/10
5628/5628 - 52s - loss: 0.0728 - accuracy: 0.9698 - val_loss: 0.1557 - val_accuracy: 0.9538
Epoch 4/10
5628/5628 - 52s - loss: 0.0608 - accuracy: 0.9755 - val_loss: 0.1697 - val_accuracy: 0.9545
Epoch 5/10
5628/5628 - 52s - loss: 0.0544 - accuracy: 0.9781 - val_loss: 0.1775 - val_accuracy: 0.9566
Epoch 6/10
5628/5628 - 52s - loss: 0.0495 - accuracy: 0.9802 - val_loss: 0.1789 - val_accuracy: 0.9505
Epoch 7/10
5628/5628 - 51s - loss: 0.0460 - accuracy: 0.9816 - val_loss: 0.2017 - val_accuracy: 0.9559
Epoch 8/10
5628/5628 - 51s - loss: 0.0438 - accuracy: 0.9824 - val_loss: 0.2060 - val_accuracy: 0.9561
Epoch 9/10
5628/5628 - 50s - loss: 0.0417 - accuracy: 0.9834 - val_loss: 0.2613 - val_accuracy: 0.956

In [14]:
try:
  from art.attacks.evasion import FastGradientMethod, Wasserstein
  from art.estimators.classification import TensorFlowV2Classifier
except ImportError:
  from sys import executable
  !{executable} -m pip install adversarial-robustness-toolbox[Keras]
  from art.attacks.evasion import FastGradientMethod, Wasserstein
  from art.estimators.classification import TensorFlowV2Classifier
import tensorflow as tf

# Train the ART classifier

def keras_classifier_art(model,nb_classes,shape):
  return TensorFlowV2Classifier(
      model=model,
      nb_classes=nb_classes,
      input_shape=shape,
      loss_object=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
  #      label_smoothing=0,
  #      axis=-1,
  #      reduction="auto",
        name="categorical_crossentropy"
    )
  )


def attack_art(discriminator, generator):
  scaler = preprocessing.StandardScaler().fit(Xtst)
  xtst = scaler.transform(Xtst).reshape(*Xtst.shape,1)
  ytst = to_categorical(Ytst, 2)

  discriminator_classifier = keras_classifier_art(discriminator,2,xtst.shape)
  predictions = discriminator_classifier.predict(xtst)
  accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(ytst, axis=1)) / len(ytst)
  print("Accuracy on benign test examples: {}%".format(accuracy * 100))

  generator_classifier = keras_classifier_art(generator,2,xtst.shape)

  # Generate adversarial test examples
  attack = FastGradientMethod(estimator=generator_classifier, eps=0.2)
  #attack = Wasserstein(estimator=classifier, eps=0.2)
  xattack = attack.generate(x=xtst)

  # Evaluate the ART classifier on adversarial test examples

  predictions = discriminator_classifier.predict(xattack)
  accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(ytst, axis=1)) / len(ytst)
  print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
  print("Mean consumption of benign data: " + str(np.mean(xtst)))
  print("Mean consumption of adversarial test examples: " + str(np.mean(xattack)))

attack_art(model, generator)


Accuracy on benign test examples: 90.24575390352807%
Accuracy on adversarial test examples: 83.97363780398081%
Mean consumption of benign data: 1.5240753471573253e-17
Mean consumption of adversarial test examples: -0.022997515
